# Scanning the BP Dataset

This tutorial demonstrates how to use **SegyIO.jl** to scan the BP 1994 2-D seismic dataset. The `segy_scan` function builds a lookup table for fast, out-of-core access to the traces. We then plot a few shot gathers and visualize the source and receiver geometry.


In [ ]:
julia
using Pkg
Pkg.add("SegyIO")
Pkg.add("PyPlot")


## Download the BP dataset

The full dataset is available from the [open source geoscience archive](http://s3.amazonaws.com/open.source.geoscience/open_data/bpmodel94/Model94_shots.segy.gz). The file is about 500&nbsp;MB compressed. Download and decompress it locally:


In [ ]:
bash
curl -O http://s3.amazonaws.com/open.source.geoscience/open_data/bpmodel94/Model94_shots.segy.gz
gunzip Model94_shots.segy.gz


## Scan the SEGY file

`segy_scan` requires the folder path, a filename filter and a list of header keys. For this dataset we scan the shot file for receiver and source coordinates:


In [ ]:
julia
using SegyIO

s = segy_scan("./", "Model94_shots.segy", ["GroupX", "GroupY", "SourceX", "SourceY"], verbosity=1)


The resulting object `s` is a `SeisCon`. It contains one block per shot record. You can query basic information such as the number of blocks and the number of samples per trace:


In [ ]:
julia
length(s)       # number of shot gathers
s.ns            # samples per trace


## Plot source positions

Source coordinates can be extracted with `get_sources`:


In [ ]:
julia
using PyPlot
srcs = get_sources(s) ./ 100.0  # scale to metres if coordinates are centimetres
figure(figsize=(6,5))
scatter(srcs[:,1], srcs[:,2], s=4)
xlabel("X")
ylabel("Y")
title("Source map")
axis("equal")


## Plot a few shot records

Select a couple of blocks from the container and read them into memory:


In [ ]:
julia
shots = read_con(s, [1, 50, 100])  # load three shot gathers
for (i, blk) in enumerate(shots.blocks)
    b = read_block!(blk, s.ns, s.dsf, Array{Float32}(undef, s.ns, 0), BinaryTraceHeader[])
    figure(figsize=(8,4))
    imshow(reverse(b.data, dims=1), aspect="auto", cmap="gray")
    xlabel("Trace")
    ylabel("Time sample")
    title("Shot gather $(i)")
end


## Receiver spread for one shot

To display receiver coordinates we only need the headers. `read_con_headers` loads them without the large trace data:


In [ ]:
julia
headers = read_con_headers(s, ["GroupX", "GroupY"], 1)
rx = get_header(headers, "GroupX")
ry = get_header(headers, "GroupY")
figure(figsize=(6,5))
scatter(rx, ry, s=4)
scatter(srcs[1,1], srcs[1,2], c="r", marker="x", label="source")
legend()
xlabel("X")
ylabel("Y")
title("Receiver positions for shot 1")
axis("equal")
